In [1]:
import pandas as pd
import numpy as np 
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv("df_dummies_m.csv")
#deleting col de sexo pois so temos masculino aqui 
df = df.drop('sexo', axis=1)
df.head()

,faixa_remuneracao_media_sm,PRODUÇÃO DE LAVOURAS TEMPORÁRIAS,HORTICULTURA E PRODUTOS DE VIVEIRO,PRODUÇÃO DE LAVOURAS PERMANENTES,PECUÁRIA,"ATIVIDADES DE SERVIÇOS RELACIONADOS COM A AGRICULTURA E A PECUÁRIA, EXCETO ATIVIDADES VETERINÁRIAS","SILVICULTURA, EXPLORAÇÃO FLORESTAL E SERVIÇOS RELACIONADOS","PESCA, AQÜICULTURA E SERVIÇOS RELACIONADOS",EXTRAÇÃO DE CARVÃO MINERAL,EXTRAÇÃO DE PETRÓLEO E GÁS NATURAL,ATIVIDADES DE SERVIÇOS RELACIONADOS COM A EXTRAÇÃO DE PETRÓLEO E GÁS EXCETO A PROSPECÇÃO REALIZADA POR TERCEIROS,EXTRAÇÃO DE MINÉRIO DE FERRO,EXTRAÇÃO DE MINERAIS METÁLICOS NÃOFERROSOS,"EXTRAÇÃO DE PEDRA, AREIA E ARGILA",EXTRAÇÃO DE OUTROS MINERAIS NÃOMETÁLICOS,ABATE E PREPARAÇÃO DE PRODUTOS DE CARNE E DE PESCADO,"PROCESSAMENTO, PRESERVAÇÃO E PRODUÇÃO DE CONSERVAS DE FRUTAS, LEGUMES E OUTROS VEGETAIS",PRODUÇÃO DE ÓLEOS E GORDURAS VEGETAIS E ANIMAIS,LATICÍNIOS,"MOAGEM, FABRICAÇÃO DE PRODUTOS AMILÁCEOS E DE RAÇÕES BALANCEADAS PARA ANIMAIS",FABRICAÇÃO E REFINO DE AÇÚCAR,TORREFAÇÃO E MOAGEM DE CAFÉ,FABRICAÇÃO DE OUTROS PRODUTOS ALIMENTÍCIOS,FABRICAÇÃO DE BEBIDAS,FABRICAÇÃO DE PRODUTOS DO FUMO,FIAÇÃO,TECELAGEM INCLUSIVE FIAÇÃO E TECELAGEM,"FABRICAÇÃO DE ARTEFATOS TÊXTEIS, INCLUINDO TECELAGEM","ACABAMENTOS EM FIOS, TECIDOS E ARTIGOS TÊXTEIS, POR TERCEIROS",FABRICAÇÃO DE ARTEFATOS TÊXTEIS A PARTIR DE TECIDOS EXCETO VESTUÁRIO E DE OUTROS ARTIGOS TÊXTEIS,FABRICAÇÃO DE TECIDOS E ARTIGOS DE MALHA,CONFECÇÃO DE ARTIGOS DO VESTUÁRIO,FABRICAÇÃO DE ACESSÓRIOS DO VESTUÁRIO E DE SEGURANÇA PROFISSIONAL,CURTIMENTO E OUTRAS PREPARAÇÕES DE COURO,FABRICAÇÃO DE ARTIGOS PARA VIAGEM E DE ARTEFATOS DIVERSOS DE COURO,FABRICAÇÃO DE CALÇADOS,DESDOBRAMENTO DE MADEIRA,"FABRICAÇÃO DE PRODUTOS DE MADEIRA, CORTIÇA E MATERIAL TRANÇADO EXCETO MÓVEIS",FABRICAÇÃO DE CELULOSE E OUTRAS PASTAS PARA A FABRICAÇÃO DE PAPEL,"FABRICAÇÃO DE PAPEL, PAPELÃO LISO, CARTOLINA E CARTÃO",FABRICAÇÃO DE EMBALAGENS DE PAPEL OU PAPELÃO,"FABRICAÇÃO DE ARTEFATOS DIVERSOS DE PAPEL, PAPELÃO, CARTOLINA E CARTÃO",EDIÇÃO,IMPRESSÃO E SERVIÇOS CONEXOS PARA TERCEIROS,REPRODUÇÃO DE MATERIAIS GRAVADOS,COQUERIAS,FABRICAÇÃO DE PRODUTOS DERIVADOS DO PETRÓLEO,ELABORAÇÃO DE COMBUSTÍVEIS NUCLEARES,FABRICAÇÃO DE PRODUTOS QUÍMICOS INORGÂNICOS,FABRICAÇÃO DE PRODUTOS QUÍMICOS ORGÂNICOS,FABRICAÇÃO DE RESINAS E ELASTÔMEROS,"FABRICAÇÃO DE FIBRAS, FIOS, CABOS E FILAMENTOS CONTÍNUOS ARTIFICIAIS E SINTÉTICOS",FABRICAÇÃO DE PRODUTOS FARMACÊUTICOS,FABRICAÇÃO DE DEFENSIVOS AGRÍCOLAS,"FABRICAÇÃO DE SABÕES, DETERGENTES, PRODUTOS DE LIMPEZA E ARTIGOS DE PERFUMARIA","FABRICAÇÃO DE TINTAS, VERNIZES, ESMALTES, LACAS E PRODUTOS AFINS",FABRICAÇÃO DE PRODUTOS E PREPARADOS QUÍMICOS DIVERSOS,FABRICAÇÃO DE ARTIGOS DE BORRACHA,FABRICAÇÃO DE PRODUTOS DE PLÁSTICO,FABRICAÇÃO DE VIDRO E DE PRODUTOS DO VIDRO,FABRICAÇÃO DE CIMENTO,"FABRICAÇÃO DE ARTEFATOS DE CONCRETO, CIMENTO, FIBROCIMENTO, GESSO E ESTUQUE",FABRICAÇÃO DE PRODUTOS CERÂMICOS,APARELHAMENTO DE PEDRAS E FABRICAÇÃO DE CAL E DE OUTROS PRODUTOS DE MINERAIS NÃOMETÁLICOS,PRODUÇÃO DE FERROGUSA E DE FERROLIGAS,SIDERURGIA,FABRICAÇÃO DE TUBOS EXCETO EM SIDERÚRGICAS,METALURGIA DE METAIS NÃOFERROSOS,FUNDIÇÃO,FABRICAÇÃO DE ESTRUTURAS METÁLICAS E OBRAS DE CALDEIRARIA PESADA,"FABRICAÇÃO DE TANQUES, CALDEIRAS E RESERVATÓRIOS METÁLICOS","FORJARIA, ESTAMPARIA, METALURGIA DO PÓ E SERVIÇOS DE TRATAMENTO DE METAIS","FABRICAÇÃO DE ARTIGOS DE CUTELARIA, DE SERRALHERIA E FERRAMENTAS MANUAIS","MANUTENÇÃO E REPARAÇÃO DE TANQUES, CALDEIRAS E RESERVATÓRIOS METÁLICOS",FABRICAÇÃO DE PRODUTOS DIVERSOS DE METAL,"FABRICAÇÃO DE MOTORES, BOMBAS, COMPRESSORES E EQUIPAMENTOS DE TRANSMISSÃO",FABRICAÇÃO DE MÁQUINAS E EQUIPAMENTOS DE USO GERAL,"FABRICAÇÃO DE TRATORES E DE MÁQUINAS E EQUIPAMENTOS PARA A AGRICULTURA, AVICULTURA E OBTENÇÃO DE PRODUTOS ANIMAIS",FABRICAÇÃO DE MÁQUINASFERRAMENTA,FABRICAÇÃO DE MÁQUINAS E EQUIPAMENTOS DE USO NA EXTRAÇÃO MINERAL E CONSTRUÇÃO,FABRICAÇÃO DE OUTRAS MÁQUINAS E EQUIPAMENTOS DE USO ESPECÍFICO,"FABRICAÇÃO DE ARMAS, MUNIÇÕES E EQUIPAMENTOS MILITARES",FABRICAÇÃO DE ELETRODOMÉSTI

## Correlacao de Pearson


In [4]:
df["faixa_remuneracao_media_sm"].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [5]:
df_primeiras_faixas=df[df['faixa_remuneracao_media_sm'].isin([0,1,2,3])] 
df_faixas_meio= df[df['faixa_remuneracao_media_sm'].isin([4,5,6,7])] 
df_ultimas_faixas= df[df['faixa_remuneracao_media_sm'].isin([ 8,  9, 10, 11])] 
dataframes = {
    "df_primeiras_faixas": df_primeiras_faixas,
    "df_faixas_meio": df_faixas_meio,
    "df_ultimas_faixas": df_ultimas_faixas,
    
}

In [6]:
dataframes = {
    "df_primeiras_faixas": df_primeiras_faixas,
    "df_faixas_meio": df_faixas_meio,
    "df_ultimas_faixas": df_ultimas_faixas,
    
}

In [7]:
df_primeiras_faixas=df[df['faixa_remuneracao_media_sm'].isin([0,1,2,3])] 
df_faixas_meio= df[df['faixa_remuneracao_media_sm'].isin([4,5,6,7])] 
df_ultimas_faixas= df[df['faixa_remuneracao_media_sm'].isin([ 8,  9, 10, 11])] 
dataframes = {
    "df_primeiras_faixas": df_primeiras_faixas,
    "df_faixas_meio": df_faixas_meio,
    "df_ultimas_faixas": df_ultimas_faixas,
    
}

for nome, dataframe in dataframes.items(): 
    # Calculando a correlação de Pearson de todas as features com a variável alvo
    correlation_matrix = dataframe.corr()

    # Extrair as correlações da variável alvo e descartar a própria variável alvo da série
    target_correlation = correlation_matrix['faixa_remuneracao_media_sm'].drop('faixa_remuneracao_media_sm')

    # Ordenar as correlações em ordem decrescente de valor absoluto para identificar as mais significativas
    top_features_correlation = target_correlation.abs().sort_values(ascending=False)
    print(f"para o {nome} as 30 variaveis mais relevantes de acordo com a correlação de Pearson sao: ")
    # Mostrando as top 10 features mais correlacionadas com a variável alvo
    print(top_features_correlation.head(30))


para o df_primeiras_faixas as 30 variaveis mais relevantes de acordo com a correlação de Pearson sao: 
ATIVIDADES DE RÁDIO E DE TELEVISÃO                                                           0.665492
COMÉRCIO A VAREJO DE COMBUSTÍVEIS                                                            0.557511
Extrativa Mineral                                                                            0.542481
Comércio Varejista                                                                           0.542392
Prod. Mineral Não Metálico                                                                   0.522633
TELECOMUNICAÇÕES                                                                             0.506515
OBRAS DE INFRAESTRUTURA PARA ENERGIA ELÉTRICA E PARA TELECOMUNICAÇÕES                        0.504389
30 A 39                                                                                      0.488476
COMÉRCIO A VAREJO E POR ATACADO DE VEÍCULOS AUTOMOTORES                          

In [8]:
##DEPOIS COMPARAR COM O RESULTADO MASCULINO PARA AMBAS AS FAIXAS 

In [9]:
for nome, dataframe in dataframes.items(): 
    X = dataframe.drop('faixa_remuneracao_media_sm', axis=1)
    y = dataframe['faixa_remuneracao_media_sm']

    chi2_features = SelectKBest(chi2, k=10)
    X_kbest_features = chi2_features.fit_transform(X, y)

    # Obtendo os scores para as features selecionadas novamente
    chi2_scores = chi2_features.scores_

    # Criando um dataframe para visualizar os scores de cada feature
    chi2_scores_df = pd.DataFrame(chi2_scores, index=X.columns, columns=["Chi2 Score"]).sort_values(by="Chi2 Score", ascending=False)
    print(f"para o {nome} as 10 variaveis mais relevantes de acordo com o Chi2 Score sao: ")
    print(chi2_scores_df.head(10))
    


para o df_primeiras_faixas as 10 variaveis mais relevantes de acordo com o Chi2 Score sao: 
                                        Chi2 Score
Comércio Varejista                   270766.298509
30 A 39                              204591.443410
CLT U/ PJ Ind                        193925.133865
Fundamental Completo                 178818.988568
FABRICAÇÃO DE ARTIGOS DO MOBILIÁRIO  175706.388660
Prod. Mineral Não Metálico           173528.339005
Adm Técnica Profissional             171122.493298
Aprendiz                             170106.221604
COMÉRCIO A VAREJO DE COMBUSTÍVEIS    164484.491436
Madeira e Mobiliário                 162545.177333
para o df_faixas_meio as 10 variaveis mais relevantes de acordo com o Chi2 Score sao: 
                                                       Chi2 Score
COMÉRCIO A VAREJO DE COMBUSTÍVEIS                   129020.633650
COMÉRCIO ATACADISTA DE PRODUTOS INTERMEDIÁRIOS ...  115807.989940
MANUTENÇÃO E REPARAÇÃO DE VEÍCULOS AUTOMOTORES      113188.426

## RFE

In [10]:

for nome, dataframe in dataframes.items(): 
    X = dataframe.drop('faixa_remuneracao_media_sm', axis=1)
    y = dataframe['faixa_remuneracao_media_sm']
    model = LinearRegression()
    rfe = RFE(estimator=model, n_features_to_select=10)
    rfe.fit(X, y)
    selected_features_rfe = X.columns[rfe.support_]
    print(f"para o {nome} as 10 variaveis mais relevantes de acordo com o rfe sao: ")
    print(selected_features_rfe)

para o df_primeiras_faixas as 10 variaveis mais relevantes de acordo com o rfe sao: 
Index(['ATIVIDADES DE SERVIÇOS RELACIONADOS COM A EXTRAÇÃO DE PETRÓLEO E GÁS  EXCETO A PROSPECÇÃO REALIZADA POR TERCEIROS',
       'EXTRAÇÃO DE MINÉRIO DE FERRO', 'FABRICAÇÃO DE CALÇADOS',
       'MANUTENÇÃO E REPARAÇÃO DE TANQUES, CALDEIRAS E RESERVATÓRIOS METÁLICOS',
       'FABRICAÇÃO DE TRATORES E DE MÁQUINAS E EQUIPAMENTOS PARA A AGRICULTURA, AVICULTURA E OBTENÇÃO DE PRODUTOS ANIMAIS',
       'RECICLAGEM DE SUCATAS METÁLICAS',
       'ALUGUEL DE OUTROS MEIOS DE TRANSPORTE', 'Extrativa Mineral',
       'Indústria Calçados', 'capital_rio_branco___ac'],
      dtype='object')
para o df_faixas_meio as 10 variaveis mais relevantes de acordo com o rfe sao: 
Index(['FABRICAÇÃO DE CIMENTO',
       'FABRICAÇÃO DE FIOS, CABOS E CONDUTORES ELÉTRICOS ISOLADOS',
       'FABRICAÇÃO DE APARELHOS, INSTRUMENTOS E MATERIAIS ÓPTICOS, FOTOGRÁFICOS E CINEMATOGRÁFICOS',
       'FABRICAÇÃO DE AUTOMÓVEIS, CAMINHONETAS E U

## lasso 

In [11]:

from sklearn.linear_model import LassoCV
for nome, dataframe in dataframes.items():
    X = dataframe.drop('faixa_remuneracao_media_sm', axis=1)
    y = dataframe['faixa_remuneracao_media_sm']
    lasso = LassoCV(cv=5).fit(X, y)
    important_features_lasso = np.where(lasso.coef_ != 0)[0]
    print(f"Para o {nome}, as variáveis mais relevantes de acordo com o Lasso são:")
    
    # Iterando sobre os índices das características importantes e imprimindo cada nome de variável em uma nova linha
    for index in important_features_lasso:
        print(X.columns[index])

/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.1803725794536981, tolerance: 0.17274971056439964
  model = cd_fast.enet_coordinate_descent_gram(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.24815990548239775, tolerance: 0.17274971056439964
  model = cd_fast.enet_coordinate_descent_gram(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.23726926818221727, tolerance: 0.1727497105643996
  model = cd_fast.enet_coordinate_descent_gram(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.

Para o df_primeiras_faixas, as variáveis mais relevantes de acordo com o Lasso são:
PECUÁRIA
MOAGEM, FABRICAÇÃO DE PRODUTOS AMILÁCEOS E DE RAÇÕES BALANCEADAS PARA ANIMAIS
FABRICAÇÃO DE BEBIDAS
FIAÇÃO
TECELAGEM  INCLUSIVE FIAÇÃO E TECELAGEM
CONFECÇÃO DE ARTIGOS DO VESTUÁRIO
FABRICAÇÃO DE PRODUTOS DE PLÁSTICO
FABRICAÇÃO DE PRODUTOS CERÂMICOS
SIDERURGIA
FABRICAÇÃO DE ARTIGOS DE CUTELARIA, DE SERRALHERIA E FERRAMENTAS MANUAIS
MANUTENÇÃO E REPARAÇÃO DE MÁQUINAS E EQUIPAMENTOS INDUSTRIAIS
FABRICAÇÃO DE ARTIGOS DO MOBILIÁRIO
PRODUÇÃO E DISTRIBUIÇÃO DE ENERGIA ELÉTRICA
PREPARAÇÃO DO TERRENO
CONSTRUÇÃO DE EDIFÍCIOS E OBRAS DE ENGENHARIA  CIVIL
OBRAS DE ACABAMENTO
COMÉRCIO A VAREJO E POR ATACADO DE VEÍCULOS AUTOMOTORES
COMÉRCIO A VAREJO E POR ATACADO DE PEÇAS E ACESSÓRIOS PARA VEÍCULOS AUTOMOTORES
COMÉRCIO A VAREJO DE COMBUSTÍVEIS
REPRESENTANTES COMERCIAIS E AGENTES DO COMÉRCIO
COMÉRCIO ATACADISTA DE PRODUTOS ALIMENTÍCIOS, BEBIDAS E FUMO
COMÉRCIO ATACADISTA DE PRODUTOS INTERMEDIÁRIOS NÃOAGROPECU

/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.19359463534496513, tolerance: 0.17274971056439958
  model = cd_fast.enet_coordinate_descent_gram(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.18826516855279607, tolerance: 0.17274971056439958
  model = cd_fast.enet_coordinate_descent_gram(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.35034101988031807, tolerance: 0.17274971056439958
  model = cd_fast.enet_coordinate_descent_gram(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descen

Para o df_faixas_meio, as variáveis mais relevantes de acordo com o Lasso são:
ABATE E PREPARAÇÃO DE PRODUTOS DE CARNE E DE PESCADO
FABRICAÇÃO DE OUTROS PRODUTOS ALIMENTÍCIOS
IMPRESSÃO E SERVIÇOS CONEXOS PARA TERCEIROS
FABRICAÇÃO DE PRODUTOS DE PLÁSTICO
FABRICAÇÃO DE PRODUTOS CERÂMICOS
SIDERURGIA
METALURGIA DE METAIS NÃOFERROSOS
FABRICAÇÃO DE ESTRUTURAS METÁLICAS E OBRAS DE CALDEIRARIA PESADA
FABRICAÇÃO DE MÁQUINAS E EQUIPAMENTOS DE USO GERAL
MANUTENÇÃO E REPARAÇÃO DE MÁQUINAS E EQUIPAMENTOS INDUSTRIAIS
FABRICAÇÃO DE MÁQUINAS E EQUIPAMENTOS DE SISTEMAS ELETRÔNICOS PARA PROCESSAMENTO DE DADOS
FABRICAÇÃO DE APARELHOS RECEPTORES DE RÁDIO E TELEVISÃO E DE REPRODUÇÃO, GRAVAÇÃO OU AMPLIFICAÇÃO DE SOM E VÍDEO
FABRICAÇÃO DE PEÇAS E ACESSÓRIOS PARA VEÍCULOS AUTOMOTORES
FABRICAÇÃO DE OUTROS EQUIPAMENTOS DE TRANSPORTE
PRODUÇÃO E DISTRIBUIÇÃO DE ENERGIA ELÉTRICA
CAPTAÇÃO, TRATAMENTO E DISTRIBUIÇÃO DE ÁGUA
PREPARAÇÃO DO TERRENO
CONSTRUÇÃO DE EDIFÍCIOS E OBRAS DE ENGENHARIA  CIVIL
OBRAS DE INFRAESTR

/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.39031373133821035, tolerance: 0.17274971056439964
  model = cd_fast.enet_coordinate_descent_gram(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.55844502980176, tolerance: 0.17274971056439964
  model = cd_fast.enet_coordinate_descent_gram(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.5650316505402202, tolerance: 0.17274971056439964
  model = cd_fast.enet_coordinate_descent_gram(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py

Para o df_ultimas_faixas, as variáveis mais relevantes de acordo com o Lasso são:
ATIVIDADES DE SERVIÇOS RELACIONADOS COM A AGRICULTURA E A PECUÁRIA, EXCETO ATIVIDADES VETERINÁRIAS
EXTRAÇÃO DE PETRÓLEO E GÁS NATURAL
ATIVIDADES DE SERVIÇOS RELACIONADOS COM A EXTRAÇÃO DE PETRÓLEO E GÁS  EXCETO A PROSPECÇÃO REALIZADA POR TERCEIROS
EDIÇÃO
FABRICAÇÃO DE PRODUTOS DERIVADOS DO PETRÓLEO
FABRICAÇÃO DE PRODUTOS QUÍMICOS INORGÂNICOS
FABRICAÇÃO DE PRODUTOS QUÍMICOS ORGÂNICOS
FABRICAÇÃO DE PRODUTOS FARMACÊUTICOS
FABRICAÇÃO DE ELETRODOMÉSTICOS
FABRICAÇÃO DE APARELHOS RECEPTORES DE RÁDIO E TELEVISÃO E DE REPRODUÇÃO, GRAVAÇÃO OU AMPLIFICAÇÃO DE SOM E VÍDEO
PRODUÇÃO E DISTRIBUIÇÃO DE ENERGIA ELÉTRICA
CONSTRUÇÃO DE EDIFÍCIOS E OBRAS DE ENGENHARIA  CIVIL
OBRAS DE INFRAESTRUTURA PARA ENERGIA ELÉTRICA E PARA TELECOMUNICAÇÕES
COMÉRCIO ATACADISTA DE ARTIGOS DE USOS PESSOAL E DOMÉSTICO
COMÉRCIO ATACADISTA DE PRODUTOS INTERMEDIÁRIOS NÃOAGROPECUÁRIOS, RESÍDUOS E SUCATAS
COMÉRCIO ATACADISTA DE MÁQUINAS, APARELHO

/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.368e+01, tolerance: 2.160e-01
  model = cd_fast.enet_coordinate_descent(


In [13]:
from sklearn.linear_model import LassoCV
for nome, dataframe in dataframes.items(): 
    X = dataframe.drop('faixa_remuneracao_media_sm', axis=1)
    y = dataframe['faixa_remuneracao_media_sm']
    lasso = LassoCV(cv=5).fit(X, y)
    important_features_lasso = np.where(lasso.coef_ != 0)[0]
    print(f"para o {nome} as 10 variaveis mais relevantes de acordo com o lasso sao: ")
    print(X.columns[important_features_lasso])

/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.1803725794536981, tolerance: 0.17274971056439964
  model = cd_fast.enet_coordinate_descent_gram(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.24815990548239775, tolerance: 0.17274971056439964
  model = cd_fast.enet_coordinate_descent_gram(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.23726926818221727, tolerance: 0.1727497105643996
  model = cd_fast.enet_coordinate_descent_gram(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.

para o df_primeiras_faixas as 10 variaveis mais relevantes de acordo com o lasso sao: 
Index(['PECUÁRIA',
       'MOAGEM, FABRICAÇÃO DE PRODUTOS AMILÁCEOS E DE RAÇÕES BALANCEADAS PARA ANIMAIS',
       'FABRICAÇÃO DE BEBIDAS', 'FIAÇÃO',
       'TECELAGEM  INCLUSIVE FIAÇÃO E TECELAGEM',
       'CONFECÇÃO DE ARTIGOS DO VESTUÁRIO',
       'FABRICAÇÃO DE PRODUTOS DE PLÁSTICO',
       'FABRICAÇÃO DE PRODUTOS CERÂMICOS', 'SIDERURGIA',
       'FABRICAÇÃO DE ARTIGOS DE CUTELARIA, DE SERRALHERIA E FERRAMENTAS MANUAIS',
       'MANUTENÇÃO E REPARAÇÃO DE MÁQUINAS E EQUIPAMENTOS INDUSTRIAIS',
       'FABRICAÇÃO DE ARTIGOS DO MOBILIÁRIO',
       'PRODUÇÃO E DISTRIBUIÇÃO DE ENERGIA ELÉTRICA', 'PREPARAÇÃO DO TERRENO',
       'CONSTRUÇÃO DE EDIFÍCIOS E OBRAS DE ENGENHARIA  CIVIL',
       'OBRAS DE ACABAMENTO',
       'COMÉRCIO A VAREJO E POR ATACADO DE VEÍCULOS AUTOMOTORES',
       'COMÉRCIO A VAREJO E POR ATACADO DE PEÇAS E ACESSÓRIOS PARA VEÍCULOS AUTOMOTORES',
       'COMÉRCIO A VAREJO DE COMBUSTÍVE

/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.19359463534496513, tolerance: 0.17274971056439958
  model = cd_fast.enet_coordinate_descent_gram(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.18826516855279607, tolerance: 0.17274971056439958
  model = cd_fast.enet_coordinate_descent_gram(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.35034101988031807, tolerance: 0.17274971056439958
  model = cd_fast.enet_coordinate_descent_gram(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descen

para o df_faixas_meio as 10 variaveis mais relevantes de acordo com o lasso sao: 
Index(['ABATE E PREPARAÇÃO DE PRODUTOS DE CARNE E DE PESCADO',
       'FABRICAÇÃO DE OUTROS PRODUTOS ALIMENTÍCIOS',
       'IMPRESSÃO E SERVIÇOS CONEXOS PARA TERCEIROS',
       'FABRICAÇÃO DE PRODUTOS DE PLÁSTICO',
       'FABRICAÇÃO DE PRODUTOS CERÂMICOS', 'SIDERURGIA',
       'METALURGIA DE METAIS NÃOFERROSOS',
       'FABRICAÇÃO DE ESTRUTURAS METÁLICAS E OBRAS DE CALDEIRARIA PESADA',
       'FABRICAÇÃO DE MÁQUINAS E EQUIPAMENTOS DE USO GERAL',
       'MANUTENÇÃO E REPARAÇÃO DE MÁQUINAS E EQUIPAMENTOS INDUSTRIAIS',
       'FABRICAÇÃO DE MÁQUINAS E EQUIPAMENTOS DE SISTEMAS ELETRÔNICOS PARA PROCESSAMENTO DE DADOS',
       'FABRICAÇÃO DE APARELHOS RECEPTORES DE RÁDIO E TELEVISÃO E DE REPRODUÇÃO, GRAVAÇÃO OU AMPLIFICAÇÃO DE SOM E VÍDEO',
       'FABRICAÇÃO DE PEÇAS E ACESSÓRIOS PARA VEÍCULOS AUTOMOTORES',
       'FABRICAÇÃO DE OUTROS EQUIPAMENTOS DE TRANSPORTE',
       'PRODUÇÃO E DISTRIBUIÇÃO DE ENERGIA EL

/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.39031373133821035, tolerance: 0.17274971056439964
  model = cd_fast.enet_coordinate_descent_gram(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.55844502980176, tolerance: 0.17274971056439964
  model = cd_fast.enet_coordinate_descent_gram(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.5650316505402202, tolerance: 0.17274971056439964
  model = cd_fast.enet_coordinate_descent_gram(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py

para o df_ultimas_faixas as 10 variaveis mais relevantes de acordo com o lasso sao: 
Index(['ATIVIDADES DE SERVIÇOS RELACIONADOS COM A AGRICULTURA E A PECUÁRIA, EXCETO ATIVIDADES VETERINÁRIAS',
       'EXTRAÇÃO DE PETRÓLEO E GÁS NATURAL',
       'ATIVIDADES DE SERVIÇOS RELACIONADOS COM A EXTRAÇÃO DE PETRÓLEO E GÁS  EXCETO A PROSPECÇÃO REALIZADA POR TERCEIROS',
       'EDIÇÃO', 'FABRICAÇÃO DE PRODUTOS DERIVADOS DO PETRÓLEO',
       'FABRICAÇÃO DE PRODUTOS QUÍMICOS INORGÂNICOS',
       'FABRICAÇÃO DE PRODUTOS QUÍMICOS ORGÂNICOS',
       'FABRICAÇÃO DE PRODUTOS FARMACÊUTICOS',
       'FABRICAÇÃO DE ELETRODOMÉSTICOS',
       'FABRICAÇÃO DE APARELHOS RECEPTORES DE RÁDIO E TELEVISÃO E DE REPRODUÇÃO, GRAVAÇÃO OU AMPLIFICAÇÃO DE SOM E VÍDEO',
       'PRODUÇÃO E DISTRIBUIÇÃO DE ENERGIA ELÉTRICA',
       'CONSTRUÇÃO DE EDIFÍCIOS E OBRAS DE ENGENHARIA  CIVIL',
       'OBRAS DE INFRAESTRUTURA PARA ENERGIA ELÉTRICA E PARA TELECOMUNICAÇÕES',
       'COMÉRCIO ATACADISTA DE ARTIGOS DE USOS PESSOAL E D

/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.368e+01, tolerance: 2.160e-01
  model = cd_fast.enet_coordinate_descent(
